# ⚡ PromptForge: The Ultimate Prompt Optimizer (Colab Edition)

Welcome to the cloud version of PromptForge! This notebook allows you to use the powerful meta-prompting engine entirely in your browser using **Google Colab**.

### Supported Backends in Colab:
- **OpenAI / Remote**: Connect to OpenAI API.
- **Anthropic**: Use Claude 3 models.
- **Google Gemini**: Use Gemini 1.5 models.
- **Groq**: Ultra-fast Llama 3 inference.

*(Note: Local backends like LM Studio and Ollama accessible via localhost are not supported here unless you set up tunneling)*

In [ ]:
# @title 1. Install Dependencies & Clone Repo
!git clone https://github.com/ahmadzkn/prompt-forge.git
%cd prompt-forge

# Install core requirements (excluding GUI libs like customtkinter which don't work in Colab)
!pip install openai anthropic google-generativeai groq sqlalchemy pydantic ipywidgets ollama llama-cpp-python

In [ ]:
# @title 2. Initialize Optimizer
import sys
import os
sys.path.append(os.getcwd())

from src.optimizer import PromptOptimizer
from src.backends.factory import ProviderFactory
from src.utils.credential_manager import CredentialManager

# Mock keyring for Colab since system keyring isn't available
import keyring
from keyring.backend import KeyringBackend
class ColabKeyring(KeyringBackend):
    def set_password(self, service, username, password):
        os.environ[f"{service}_{username}"] = password
    def get_password(self, service, username):
        return os.environ.get(f"{service}_{username}")
    def delete_password(self, service, username):
        pass
    def priority(self):
        return 1

keyring.set_keyring(ColabKeyring())

In [ ]:
# @title 3. Run PromptForge UI
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

# Widgets
backend_dropdown = widgets.Dropdown(
    options=['OpenAI', 'Anthropic', 'Google Gemini', 'Groq'],
    value='Groq',
    description='Backend:',
)

api_key_input = widgets.Password(
    placeholder='Enter API Key',
    description='API Key:',
)

model_input = widgets.Text(
    value='',
    placeholder='Model Name (e.g. llama3-70b-8192)',
    description='Model:',
)

raw_prompt_area = widgets.Textarea(
    placeholder='Enter your raw prompt here...',
    description='Raw Prompt:',
    layout=widgets.Layout(width='100%', height='150px')
)

optimize_btn = widgets.Button(
    description='Optimize Prompt',
    button_style='primary',
    icon='magic'
)

output_area = widgets.Output()

def on_optimize(b):
    with output_area:
        clear_output()
        print("Optimizing... Please wait.")
        
        backend_map = {
            'OpenAI': 'openai',
            'Anthropic': 'anthropic',
            'Google Gemini': 'gemini',
            'Groq': 'groq'
        }
        
        provider_type = backend_map[backend_dropdown.value]
        api_key = api_key_input.value
        model = model_input.value
        raw_prompt = raw_prompt_area.value
        
        if not api_key:
            print("Error: Please enter an API Key.")
            return
            
        try:
            kwargs = {"api_key": api_key}
            if provider_type == 'openai':
                kwargs['base_url'] = "https://api.openai.com/v1" # Default to real OpenAI
            
            optimizer = PromptOptimizer(provider_type=provider_type, **kwargs)
            
            # If model is empty, try to auto-fetch or set default
            if not model:
                models = optimizer.get_available_models()
                if models:
                    model = models[0]
                    print(f"Auto-selected model: {model}")
                else:
                    print("Error: Could not fetch models. Please specify one manually.")
                    return
            
            result = optimizer.optimize_prompt(raw_prompt, model)
            
            clear_output()
            
            if "error" in result:
                print(f"Error: {result['error']}")
                return
                
            print(f"### Final Optimized Prompt (Backend: {backend_dropdown.value} / {model})")
            print("-" * 80)
            print(result.get("final_prompt", "No result"))
            print("-" * 80)
            
            print("\n### Structured Elements:")
            elements = result.get("elements", {})
            for k, v in elements.items():
                print(f"**{k.title()}**: {v}")

        except Exception as e:
            print(f"Unexpected Error: {e}")

optimize_btn.on_click(on_optimize)

display(widgets.VBox([
    widgets.HTML("<h2>PromptForge Configuration</h2>"),
    backend_dropdown,
    api_key_input,
    model_input,
    widgets.HTML("<h2>Input</h2>"),
    raw_prompt_area,
    optimize_btn,
    widgets.HTML("<h2>Output</h2>"),
    output_area
]))